In [1]:
import sys

In [2]:
import findspark
findspark.init()
import pyspark
import random

In [3]:
from elasticsearch import Elasticsearch

In [6]:
es = Elasticsearch()

#### Run the follwing command in the terminal(to authenticate) to get the mail archive before running the next commands
gmvault sync you@gmail.com -d path/to/where/you/want/the/email/archived

In [11]:
!cat /Users/GodSpeed/Documents/mailArchive/db/2010-10/1349228336576455316.meta

{"thread_ids": 1349228336576455316, "msg_id": "1785262635.1286724410050.JavaMail.async@sl-prod-sso-eas5-1", "flags": ["\\Seen"], "gm_id": 1349228336576455316, "internal_date": 1286706411, "x_gmail_received": null, "labels": ["\\Inbox"], "subject": "Welcome to your Nokia account"}

In [16]:
!gzcat /Users/GodSpeed/Documents/mailArchive/db/2010-10/1349228336576455316.eml.gz

### ===================================
We’ll walk all the files in the root of the Gmvault database using os.walk, find all files that end in .meta, load the JSON in those files, tweak the JSON just a bit (more on that in a second), and then shove the JSON into Elasticsearch.

In [4]:
import os, json, timeit

In [57]:
#can just store the mapping keys instead of the whole document,and when searched, 
#may access the data using the mapping key. This may be doe by storing the actual data somewhere else, 
#maybe hadoop or not

def fileToES(folder_path):
    for parent_folder, folders, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith('.meta'):
                with open(os.path.join(parent_folder, filename), 'r') as fp:
                    meta = json.load(fp)
                #add additional attributes to meta[_'source'], 
                #all new additions are made implicitly inside the meta['_source']
                meta['account'] = 'rohit.mlhtra@gmail.com'
                meta['path'] = os.path.join(parent_folder, filename)
                es.index(index = 'mail', doc_type = 'message', id = meta['gm_id'], body=meta)

In [60]:
folder_path = '/Users/GodSpeed/Documents/mailArchive/'
fileToES(folder_path)

## Searching the metadata

##### Lazy search(non structured query)

In [9]:
es.search('mail', q='playstation')

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1574119758260325826',
    u'_index': u'mail',
    u'_score': 9.661735,
    u'_source': {u'account': u'rohit.mlhtra@gmail.com',
     u'flags': [u'\\Seen'],
     u'gm_id': 1574119758260325826,
     u'internal_date': 1501179584,
     u'labels': [u'\\Inbox'],
     u'msg_id': u'0100015d86594699-8fb526c7-a6dc-4bfd-9c3f-7ea2c9febfd5-000000@email.amazonses.com',
     u'path': u'/Users/GodSpeed/Documents/mailArchive/db/2017-07/1574119758260325826.meta',
     u'subject': u'PlayStation 4 Slim 500GB...',
     u'thread_ids': 1574119758260325826,
     u'x_gmail_received': None},
    u'_type': u'message'},
   {u'_id': u'1585158307223876652',
    u'_index': u'mail',
    u'_score': 9.435886,
    u'_source': {u'account': u'rohit.mlhtra@gmail.com',
     u'flags': [u'\\Seen'],
     u'gm_id': 1585158307223876652,
     u'internal_date': 1511706764,
     u'labels': [u'\\Inbox'],
     u'msg_id': u'c40d64

#### To do: 
1. sync offline mails regularly